In [1]:
from __future__ import absolute_import, division, print_function

# Import TensorFlow >= 1.10 and enable eager execution
import tensorflow as tf

tf.enable_eager_execution()

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import time
import pandas as pd

print(tf.__version__)

1.10.0


In [2]:
#importing libraries 
import pandas as pd
import numpy as np 
import re
import string
import unicodedata

import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.layers.embeddings import Embedding
from keras import backend as K
from keras.callbacks import ModelCheckpoint 

Using TensorFlow backend.


In [3]:
#Loading the data 
french_sentences = pd.read_csv(r"s3://cognitivecomputingbucket/french_100000.csv") 
english_sentences = pd.read_csv(r"s3://cognitivecomputingbucket/english_100000.csv")

In [4]:
#takign only sentence columns 
english_sentences = english_sentences.iloc[:,-1:]
french_sentences = french_sentences.iloc[:,-1:]
print(english_sentences.columns)
print(french_sentences.columns)

Index(['English_sentences'], dtype='object')
Index(['French_sentences'], dtype='object')


In [5]:
#Taking 1000 sample of sentences 
n = 1000
french_sample = french_sentences.iloc[:n,:]
english_sample = english_sentences.iloc[:n,:]

In [6]:
#Takign only sentences which are strings 
english_sample.columns
english_sample['English_sentences']
for i in range(len(english_sample['English_sentences'].index)):
    if type(english_sample['English_sentences'][i]) != str:
        english_sample['English_sentences'][i] = str(english_sample['English_sentences'][i])
        
french_sample['French_sentences']
for i in range(len(french_sample['French_sentences'].index)):
    if type(french_sample['French_sentences'][i]) != str:
        french_sample['French_sentences'][i] = str(french_sample['French_sentences'][i])

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
#Removing the spaces---Stripping the sentence
#making the sentence lower case
#Appending each sentence with start and end keyword
def preprocess_sentence(w):
    w = w.lower().strip()
    w = '<start> ' + w + ' <end>'
    return w

#Applying the pre-processing steps to all the sentences
def cleaning_sentence(data, column):
    sentence = [preprocess_sentence(i) for i in data[column]]
    return sentence


In [8]:
#Creating the class to get sentence indexex which has word to index and index to word
class LanguageIndex():    
    def __init__(self, lang):        
        self.lang = lang
        self.word2idx = {}
        self.idx2word = {}
        self.vocab = set()
        self.create_index()
        
    def create_index(self):
        for phrase in self.lang:
          self.vocab.update(phrase.split(' '))

        self.vocab = sorted(self.vocab)

        self.word2idx['<pad>'] = 0
        for index, word in enumerate(self.vocab):
          self.word2idx[word] = index + 1

        for word, index in self.word2idx.items():
          self.idx2word[index] = word   
    


In [9]:
#finding the maximum lenght of sentences
def max_length(tensor):
    return max(len(t) for t in tensor)

In [10]:
#Cleaning Sentences
french_sent = cleaning_sentence(french_sample, 'French_sentences')
english_sent = cleaning_sentence(english_sample, 'English_sentences')

# index language using the class defined above
inp_lang = LanguageIndex(en for en in english_sent)
targ_lang = LanguageIndex(fr for fr in french_sent)

#English sentences
input_tensor = [[inp_lang.word2idx[s] for s in en.split(' ')] for en in english_sent]
#French Sentences
target_tensor = [[targ_lang.word2idx[s] for s in fr.split(' ')] for fr in french_sent]

max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)

# Padding the input and output tensor to the maximum length
input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor, 
                                                             maxlen=max_length_inp,
                                                             padding='post')

target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor, 
                                                              maxlen=max_length_tar, 
                                                              padding='post')




    
    
    

In [11]:
#Splitting the sentence train and test split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

In [12]:
# Show length of input and validation sentences
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

(800, 800, 200, 200)

In [13]:
BUFFER_SIZE = len(input_tensor_train) #Buffer size
BATCH_SIZE = 64 #Batch size 
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 256 #Embedding dimension
units = 1024 #GRU units  
vocab_inp_size = len(inp_lang.word2idx) #Vocabulary size of English sentences
vocab_tar_size = len(targ_lang.word2idx)#Vocabulary size of french sentences


print(BUFFER_SIZE)
print(BATCH_SIZE)
print(N_BATCH)
print(vocab_inp_size)
print(vocab_tar_size)



800
64
12
3343
4290


In [14]:
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 117), (64, 135)), types: (tf.int32, tf.int32)>

In [15]:
#GRU layer
def gru(units):
    return tf.keras.layers.GRU(units, 
                           return_sequences=True, 
                           return_state=True, 
                           recurrent_activation='sigmoid', 
                           recurrent_initializer='glorot_uniform')



In [16]:
#Encoder layer
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.enc_units)
        
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)        
        return output, state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [17]:
#Decoder Layer
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.dec_units)
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        # used for attention
        self.W1 = tf.keras.layers.Dense(self.dec_units)
        self.W2 = tf.keras.layers.Dense(self.dec_units)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
      
        #Scores calculations
        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))
        
        # attention_weights shape---applying softmax
        attention_weights = tf.nn.softmax(score, axis=1)
        
        # context_vector calculation
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        # passing the concatenated vector to the GRU
        output, state = self.gru(x)
        
        
        output = tf.reshape(output, (-1, output.shape[2]))     
       
        x = self.fc(output)
        
        return x, state, attention_weights

In [18]:
#intialize the hidden states of encoder 
def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.dec_units))

In [19]:
#Encoder and decoder layer 
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

In [20]:
#Optimizer function--Adam optimizer 
optimizer = tf.train.AdamOptimizer()
#Loss sparse softmax cross entropy with logit 
def loss_function(real, pred):
    mask = 1 - np.equal(real, 0)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
    return tf.reduce_mean(loss_)

In [ ]:
#Training the model for 50 epochs 
EPOCHS = 50
for epoch in range(EPOCHS):
    start = time.time()
    #Hidden states initialization
    hidden = encoder.initialize_hidden_state()
    total_loss = 0
    #epoch starts
    for (batch, (inp, targ)) in enumerate(dataset):
        loss = 0
        
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(inp, hidden)
            
            dec_hidden = enc_hidden
            
            dec_input = tf.expand_dims([targ_lang.word2idx['<start>']] * BATCH_SIZE, 1)       
            
            # Teacher forcing - feeding the target as the next input
            for t in range(1, targ.shape[1]):
                # passing enc_output to the decoder
                predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
                
                loss += loss_function(targ[:, t], predictions)
                
                # using teacher forcing
                dec_input = tf.expand_dims(targ[:, t], 1)
        
        batch_loss = (loss / int(targ.shape[1]))
        
        total_loss += batch_loss
        
        variables = encoder.variables + decoder.variables
        
        gradients = tape.gradient(loss, variables)
        
        optimizer.apply_gradients(zip(gradients, variables))
        
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))   
    
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / N_BATCH))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 1.9022
Epoch 1 Loss 1.5253
Time taken for 1 epoch 362.0095627307892 sec

Epoch 2 Batch 0 Loss 1.5867
Epoch 2 Loss 1.4084
Time taken for 1 epoch 359.5380024909973 sec

Epoch 3 Batch 0 Loss 1.3944
Epoch 3 Loss 1.3854
Time taken for 1 epoch 364.6735143661499 sec

Epoch 4 Batch 0 Loss 1.3632
Epoch 4 Loss 1.3727
Time taken for 1 epoch 363.55119585990906 sec

Epoch 5 Batch 0 Loss 1.4961
Epoch 5 Loss 1.3799
Time taken for 1 epoch 362.4090211391449 sec

Epoch 6 Batch 0 Loss 1.1157
Epoch 14 Loss 1.1711
Time taken for 1 epoch 363.65593552589417 sec

Epoch 15 Batch 0 Loss 1.1292
Epoch 15 Loss 1.1334
Time taken for 1 epoch 361.9164071083069 sec

Epoch 16 Batch 0 Loss 0.9567
Epoch 16 Loss 1.1213
Time taken for 1 epoch 376.57103967666626 sec

Epoch 17 Batch 0 Loss 1.1284
Epoch 17 Loss 1.0882
Time taken for 1 epoch 359.595495223999 sec

Epoch 18 Batch 0 Loss 1.2492
Epoch 18 Loss 1.0711
Time taken for 1 epoch 367.5085361003876 sec

Epoch 19 Batch 0 Loss 0.9386
Epoch 19 Loss 1.0457

In [26]:
#Evaluating the model
def evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    
    sentence = preprocess_sentence(sentence)

    inputs = [inp_lang.word2idx[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    
    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word2idx['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        
        # storing the attention weigths to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.idx2word[predicted_id] + ' '

        if targ_lang.idx2word[predicted_id] == '<end>':
            return result, sentence, attention_plot
        
        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

In [27]:
#Translating the predicted sentences
def translate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    result, sentence, attention_plot = evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)
        
    print('Input: {}'.format(sentence))
    print('Predicted translation: {}'.format(result))
    

In [29]:
translate(english_sample.iloc[:1,:].values[0][0], encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_tar)


Input: <start> resumption of the session <end>
Predicted translation: il sagit de la commission <end> 
